In [1]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing as pp
from skimage import exposure
from skimage.util.dtype import dtype_range
import sys
from scipy.ndimage.filters import gaussian_filter


os.environ['OPENCV_IO_ENABLE_JASPER']= 'TRUE'

In [2]:
#処理関数
def eq(img):
    # Equalization
    img_eq = ((exposure.equalize_hist(img.reshape(-1)).reshape(img.shape))*255).astype(np.uint8)
    return img_eq
def re_in(img):
        # Contrast stretching
    p2, p99 = np.percentile(img, (2, 99))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p99)).astype(np.uint8)
    return img_rescale
def ch(img):#kernel_size is 1/8 of image height by 1/8 of its width.
    # clahe
    img_ch = ((exposure.equalize_adapthist(img))*255).astype(np.uint8)#,kernel_size=[8,8]
    return img_ch
def cn(b):
    b2=pp.minmax_scale(b.reshape(-1)).reshape(b.shape)
    b3=(b2*255).astype(np.uint8)
    return b3

def re_in_sigmoid4(img):
    p2, p98 = np.percentile(img, (4, 96))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    img_rescale_sigmoid = exposure.adjust_sigmoid(img_rescale).astype(np.uint8)
    return img_rescale_sigmoid



def sigmoid(img):
    img_sigmoid = exposure.adjust_sigmoid(img).astype(np.uint8)
    return img_sigmoid

def rb(b):
    scaler=pp.RobustScaler(with_centering=True, with_scaling=True, 
                           quantile_range=(3.0, 98.0), copy=True)
    b=b.reshape(b.shape[0],-1)
    b3=scaler.fit(b)
    b3=b3.astype(np.uint8).reshape(b.shape)
    return b3

def nikon15(a):
    b = cv2.blur(a,(15,15))#ノイズ除去？
    c = a/b #ウェルの縁のみ抽出
    d= c *(a.mean()/c.mean())
    #e =a-d
    #f =e *(a.mean()/e.mean())
    
    #print(a.mean())
    #print(c.mean())
    #print(d.mean())
    #print(e.mean())
    #print(a.mean())
    #e=e.astype(np.uint8)
    return d.astype(np.uint8)

def nikon10(a):
    b = cv2.blur(a,(10,10))#ノイズ除去？
    c = a/b #ウェルの縁のみ抽出
    d= c *(a.mean()/c.mean())
    #e =a-d
    #f =e *(a.mean()/e.mean())
    
    #print(a.mean())
    #print(c.mean())
    #print(d.mean())
    #print(e.mean())
    #print(a.mean())
    #e=e.astype(np.uint8)
    return d.astype(np.uint8)

def nikon8(a):
    b = cv2.blur(a,(8,8))#ノイズ除去？
    c = a/b #ウェルの縁のみ抽出
    d= c *(a.mean()/c.mean())
    #e =a-d
    #f =e *(a.mean()/e.mean())
    
    #print(a.mean())
    #print(c.mean())
    #print(d.mean())
    #print(e.mean())
    #print(a.mean())
    #e=e.astype(np.uint8)
    return d.astype(np.uint8)

def nikon5(a):
    b = cv2.blur(a,(5,5))#ノイズ除去？
    c = a/b #ウェルの縁のみ抽出
    d= c *(a.mean()/c.mean())
    #e =a-d
    #f =e *(a.mean()/e.mean())
    
    #print(a.mean())
    #print(c.mean())
    #print(d.mean())
    #print(e.mean())
    #print(a.mean())
    #e=e.astype(np.uint8)
    return d.astype(np.uint8)

def nikon130(a):
    b = cv2.blur(a,(15,15))#ノイズ除去？
    c = a/b #ウェルの縁のみ抽出
    d= c *(130/c.mean())
    return d.astype(np.uint8)

def nikon10_130(a):
    b = cv2.blur(a,(10,10))#ノイズ除去？
    c = a/b #ウェルの縁のみ抽出
    d= c *(130/c.mean())
    return d.astype(np.uint8)

In [3]:
#フィルター処理
#blur = cv2.blur(img,(5,5))　#カーネルの画素の平均を出力
#blur = cv2.GaussianBlur(img,(5,5),0)　#カーネルは小さく無いと特徴が薄くなりそう（？）
#median = cv2.medianBlur(img,5)　#ごま塩ノイズに有効、カーネル内の中央値を出力
#blur = cv2.bilateralFilter(img,9,75,75) #エッジを保存するガウシアン

In [4]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [5]:
print('x_train.shape', x_train.shape)
# ('x_train.shape', (50000, 32, 32, 3))

x_train = x_train.astype('float32') / 255

NameError: name 'x_train' is not defined

In [6]:
def normalizeImage2(x, epsilon=1E-6):
    vmin = np.min(x)
    vmax = np.max(x)
    return (x - vmin) / (vmax - vmin + epsilon)

class PCAWhitening:
    def __init__(self, epsilon=1E-6):
        self.epsilon = epsilon
        self.mean = None
        self.eigenvalue = None
        self.eigenvector = None
        self.pca = None
    def fit(self, x):
        self.mean = np.mean(x, axis=0)
        x_ = x - self.mean
        cov = np.dot(x_.T, x_) / x_.shape[0]
        E, D, _ = np.linalg.svd(cov)
        D = np.sqrt(D) + self.epsilon
        self.eigenvalue = D
        self.eigenvector = E
        self.pca = np.dot(np.diag(1. / D), E.T)
        return self
            
    def transform(self, x):
        x_ = x - self.mean
        return np.dot(x_, self.pca.T)

In [7]:
class ZCAWhitening:
    def __init__(self, epsilon=1E-6):
        self.epsilon = epsilon
        self.mean = None
        self.zca = None

    def fit(self, x):
        self.mean = np.mean(x, axis=0)
        x_ = x - self.mean
        cov = np.dot(x_.T, x_) / x_.shape[0]
        E, D, _ = np.linalg.svd(cov)
        D = np.sqrt(D) + self.epsilon
        self.zca = np.dot(E, np.dot(np.diag(1.0 / D), E.T))
        return self

    def transform(self, x):
        x_ = x - self.mean
        return np.dot(x_, self.zca.T)

In [8]:
x_pcaw = x_train.reshape(x_train.shape[0], -1)
print('x_pcaw.shape=' + str(x_pcaw.shape))
# x_pcaw.shape(50000, 3072)
pcaw = PCAWhitening().fit(x_pcaw)
x_pcaw = pcaw.transform(x_pcaw).reshape(x_train.shape)

normalizeImage2(x_pcaw[i,:,:,:])

NameError: name 'x_train' is not defined

In [9]:
dir='/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/image_process/JPEGImages_rotate0916'
save_dir="/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/normalize_method200605/"

files=glob.glob(dir+"/*.jp2")

os.makedirs(save_dir+"JPEGImages_rotate0916_nonikon_nk10_PCAwhitening/",exist_ok=True)
os.makedirs(save_dir+"JPEGImages_rotate0916_nonikon_nk10_ZCAwhitening/",exist_ok=True)

os.makedirs(save_dir+"JPEGImages_rotate0916_nonikon_nk10_130_eq/",exist_ok=True)


print(len(files))

for file in files[25450:]:
    f=cv2.imread(file)
    name=os.path.basename(file)       
    f2=nikon10(f)
    x_pcaw = f2.reshape(f2.shape[0], -1)
    print('x_pcaw.shape=' + str(x_pcaw.shape))
    # x_pcaw.shape(50000, 3072)
    pcaw = PCAWhitening().fit(x_pcaw)
    x_pcaw = pcaw.transform(x_pcaw).reshape(f2.shape)
    cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_PCAwhitening/"+name,normalizeImage2(x_pcaw))
    
    x_zcaw=f2.reshape(f2.shape[0], -1)
    zcaw = ZCAWhitening().fit(x_zcaw)
    x_zcaw = zcaw.transform(x_zcaw).reshape(f2.shape)
    cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_ZCAwhitening/"+name,normalizeImage2(x_pcaw))
    print(x_pcaw.shape)
    
    cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_130_eq/"+name,eq(nikon10_130(f)))


for file in files[:25455]:
    f=cv2.imread(file)
    name=os.path.basename(file)
    
    cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_130_eq/"+name,eq(nikon10_130(f)))
    
    
    
    

49972
x_pcaw.shape=(511, 1536)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in true_divide


(511, 512, 3)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in true_divide


x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)


...

x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)


(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shap

x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)


(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shape=(511, 1536)
(511, 512, 3)
x_pcaw.shap

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
dir='/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/image_process/JPEGImages_rotate0916'
save_dir="/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/normalize_method200605/"
files=glob.glob(dir+'/*.jp2')
les_PCA_files=glob.glob('/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/normalize_method200605/JPEGImages_rotate0916_nonikon_nk10_PCAwhitening/*.jp2')
les_130_files=glob.glob('/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/normalize_method200605/JPEGImages_rotate0916_nonikon_nk10_130_eq/*.jp2')

a=[]
b=[]
c=[]

for i in files:
    a.append(os.path.basename(i))
    
for i in les_PCA_files:
    b.append(os.path.basename(i))
    
for i in les_130_files:
    c.append(os.path.basename(i))
    
print(b[:10])



['20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY001C1_2-color0-f0.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY001C1_2-color0-f1.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY001C1_2-color0-rotate_180.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY001C1_2-color0-rotate_270.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY001C1_2-color0-rotate_90.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY001C1_2-color0.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0-color0-f0.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0-color0-f1.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0-color0-rotate_180.jp2', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0-color0-rotate_270.jp2']


In [16]:
ab_diff = set(a) ^ set(b)
ac_diff = set(a) ^ set(c)

ab_diff=list(ab_diff)
ac_diff=list(ac_diff)

print(len(ab_diff))
print(len(ac_diff))


1
0


In [12]:
for file in ac_diff:
    f=cv2.imread(dir+'/'+file)
    name=os.path.basename(dir+'/'+file)
    
    cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_130_eq/"+name,eq(nikon10_130(f)))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:108: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in multiply


In [17]:
count=0
for file in ab_diff[1:]:
    f=cv2.imread(dir+'/'+file)
    name=file
    
    f2=nikon10(f)
    x_pcaw = f2.reshape(f2.shape[0], -1)
    # x_pcaw.shape(50000, 3072)
    try:
        pcaw = PCAWhitening().fit(x_pcaw)

        x_pcaw = pcaw.transform(x_pcaw).reshape(f2.shape)
        cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_PCAwhitening/"+name,normalizeImage2(x_pcaw))

        x_zcaw=f2.reshape(f2.shape[0], -1)
        zcaw = ZCAWhitening().fit(x_zcaw)
        x_zcaw = zcaw.transform(x_zcaw).reshape(f2.shape)
        cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_ZCAwhitening/"+name,normalizeImage2(x_pcaw))
        
    except:
        cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_PCAwhitening/"+name,f2)
        cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk10_ZCAwhitening/"+name,f2)        
        count+=1
        continue

print('error 数は'+str(count))

error 数は0


In [4]:
#0617
dir='/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/image_process/JPEGImages_rotate0916'
save_dir="/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/normalize_method200605/"

files=glob.glob(dir+"/*.jp2")

os.makedirs(save_dir+"JPEGImages_rotate0916_nonikon_nk15_eq/",exist_ok=True)


print(len(files))

for file in files:
    f=cv2.imread(file)
    name=os.path.basename(file)       
    cv2.imwrite(save_dir+"JPEGImages_rotate0916_nonikon_nk15_eq/"+name,eq(nikon15(f)))           


49972


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'script', 'normalize_execution_0617.ipynb'])